In [19]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [2]:
odds2018 = '/Users/sec/galvanize/bracket_buster/odds_data/ncaa_basketball_2017-18.xlsx'
odds2017 = '/Users/sec/galvanize/bracket_buster/odds_data/ncaa_basketball_2016-17.xlsx'
odds2016 = '/Users/sec/galvanize/bracket_buster/odds_data/ncaa_basketball_2015-16.xlsx'
odds2015 = '/Users/sec/galvanize/bracket_buster/odds_data/ncaa_basketball_2014-15.xlsx'
odds2014 = '/Users/sec/galvanize/bracket_buster/odds_data/ncaa_basketball_2013-14.xlsx'
odds2013 = '/Users/sec/galvanize/bracket_buster/odds_data/ncaa_basketball_2012-13.xlsx'

In [3]:
odds2018_df = pd.read_excel(odds2018, header=0)
odds2017_df = pd.read_excel(odds2017, header=0)
odds2016_df = pd.read_excel(odds2016, header=0)
odds2015_df = pd.read_excel(odds2015, header=0)
odds2014_df = pd.read_excel(odds2014, header=0)
odds2013_df = pd.read_excel(odds2013, header=0)

In [4]:
odds2018_df[(odds2018_df.index == 5514) | (odds2018_df.index == 5515)]

,Date,Rot,VH,Team,1st,2nd,Final,Open,Close,ML,2H
5514,207,735,V,St.Johns,39,40,79,152,151,NaN,80
5515,207,736,H,Villanova,34,41,75,18,16,pk,11


In [5]:
def prob(row):
    if type(row['ML']) != int:
        row['p'] = 'impute'
    elif row['ML'] < 0:
        row['p'] = int(row['ML']) / int((row['ML']) - 100)
    elif row['ML'] > 0:
        row['p'] = 100 / int((row['ML']) + 100)

    return row

In [6]:
def target(row):
    #need to impute Nans first
    if type(row['ML']) != int:
        row['p'] = 'impute'
    elif row['ML'] < 0:
        row['p'] = int(row['ML']) / int((row['ML']) - 100)
    elif row['ML'] > 0:
        row['p'] = 100 / int((row['ML']) + 100)

    return row

In [7]:
def spread(row):
    pass

In [8]:
def matchup(row):
    pass

In [9]:
def matchups(df, season):
    
    # Drop uneeded columns
    df = df.drop(['1st', '2H', '2nd'], axis=1)
    
    # Add Season
    df['Season'] = season
    
    # Add probability of winning column
    df = df.apply(prob, axis=1)
    
    # One hot encode VH column for counting
    df['VHohe'] = df['VH'].map({'V': 1, 'H': 0})
    
    # Create count column to use as merge ID
    df['count'] = df.groupby('VHohe').cumcount() + 1
    
    # Split df in to visitor and home team dfs
    df_v = df[df['VH'] == 'V']
    df_h = df[df['VH'] == 'H']
    
    # update column names for visitors df
    v_cols = df_v.columns.tolist()
    v_cols = ['{}_v'.format(col) if col != 'count' else col for col in v_cols]
    df_v.columns = v_cols
    
    # Merge on count
    df = pd.merge(df_h, df_v, how='left', on='count')
    
    # Drop uneeded columns
    df = df.drop(['Rot', 'Season_v', 'VH', 'VH_v', 'Date_v', 'Rot_v'], axis=1)
    
    # spread
    
    return df

In [10]:
odds2018_df = matchups(odds2018_df, 2018)
# odds2017_df = odds2017_df.apply(prob, axis=1)
# odds2016_df = odds2016_df.apply(prob, axis=1)
# odds2015_df = odds2015_df.apply(prob, axis=1)
# odds2014_df = odds2014_df.apply(prob, axis=1)
# odds2013_df = odds2013_df.apply(prob, axis=1)

In [11]:
odds2018_df.head()

,Date,Team,Final,Open,Close,ML,Season,p,VHohe,count,Team_v,Final_v,Open_v,Close_v,ML_v,p_v,VHohe_v
0,1110,WestVirginia,65,6,7,-330,2018,0.767442,0.0,1,TexasA&M,88,141,143,260,0.277778,1.0
1,1110,Duke,97,18.5,19.5,-3750,2018,0.974026,0.0,2,Elon,68,155.5,156.5,1875,0.0506329,1.0
2,1110,Richmond,63,10.5,9.5,-550,2018,0.846154,0.0,3,Delaware,76,143.5,140.5,400,0.2,1.0
3,1110,OldDominion,57,4.5,5,-250,2018,0.714286,0.0,4,Towson,54,131,142,200,0.333333,1.0
4,1110,LoyolaChicago,84,12,12.5,-950,2018,0.904762,0.0,5,WrightState,80,142.5,138,625,0.137931,1.0


To Do:

1. update Date format
4. impute NaNs with Linear Regression
5. Figure out how to merge into gamelog data
6. how many games are picked correctly jsut by favorites (am I better?)
7. Does this new p feature help my predictive model?

In [12]:
odds2018_df.columns

Index(['Date', 'Team', 'Final', 'Open', 'Close', 'ML', 'Season', 'p', 'VHohe',
       'count', 'Team_v', 'Final_v', 'Open_v', 'Close_v', 'ML_v', 'p_v',
       'VHohe_v'],
      dtype='object')

In [13]:
odds2018_df.head()

,Date,Team,Final,Open,Close,ML,Season,p,VHohe,count,Team_v,Final_v,Open_v,Close_v,ML_v,p_v,VHohe_v
0,1110,WestVirginia,65,6,7,-330,2018,0.767442,0.0,1,TexasA&M,88,141,143,260,0.277778,1.0
1,1110,Duke,97,18.5,19.5,-3750,2018,0.974026,0.0,2,Elon,68,155.5,156.5,1875,0.0506329,1.0
2,1110,Richmond,63,10.5,9.5,-550,2018,0.846154,0.0,3,Delaware,76,143.5,140.5,400,0.2,1.0
3,1110,OldDominion,57,4.5,5,-250,2018,0.714286,0.0,4,Towson,54,131,142,200,0.333333,1.0
4,1110,LoyolaChicago,84,12,12.5,-950,2018,0.904762,0.0,5,WrightState,80,142.5,138,625,0.137931,1.0


In [27]:
def line(row):
    home = []
    if type(row['Open']) == int:
        home.append(row['Open'])
    elif type(row['Close']) == int:
        home.append(row['Close'])
        
    if len(home) == 0:
        row['line'] = 'drop'
    else:
        row['line'] = np.mean(home)
    
    
    away = []
    if type(row['Open_v']) == int:
        away.append(row['Open_v'])
    elif type(row['Close_v']) == int:
        away.append(row['Close_v'])
        
    if len(away) == 0:
        row['line_v'] = 'drop'
    else:
        row['line_v'] = np.mean(away)
    return row

In [22]:
def imput_nans(df):
    train_df = df[df['p'] != 0.0]
    test_df = df[df['p'] == 0.0]
    
    Xy_train = train_df[['p', 'Open', 'Close', 'Open_v', 'Close_v']]
    Xy_test = test_df[['p', 'Open', 'Close', 'Open_v', 'Close_v']]
    
#     Xy_train = 
    
#     # Set up features and targets
#     X_train = Xy_train.iloc[:, 1:].as_matrix()
#     y_train = Xy_train.iloc[:, 0].as_matrix()
#     X_test = Xy_test.iloc[:, 1:].as_matrix()
#     y_test = Xy_test.iloc[:, 0].as_matrix()
#     
#     pipeline = Pipeline(steps=[('Standard Scaler', StandardScaler()),
#                                ('Linear Regession', LinearRegression())
#                               ])
#     
#     pipeline.fit(X_train, y_train)
#     
#     rmse = cross_val_score(pipeline, X_train, y=y_train, scoring='mse', cv=5, n_jobs=-1, verbose=1)
# 
#     predicitons = pipeline.predict(X_test)
#     score = metrics.accuracy_score(y_test, predicitons)
#     
#     return rmse
    return Xy_train
    

In [29]:
Xy_train_df = imput_nans(odds2018_df)

In [30]:
Xy_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3469 entries, 0 to 3468
Data columns (total 5 columns):
p          3469 non-null object
Open       3460 non-null object
Close      3467 non-null object
Open_v     3413 non-null object
Close_v    3456 non-null object
dtypes: object(5)
memory usage: 162.6+ KB
